Преподаватель: Александр Кондрашкин  
Использовать dataset MovieLens  
Построить рекомендации (регрессия, предсказываем оценку) на фичах:    
- TF-IDF на тегах и жанрах
- Средние оценки (+ median, variance, etc.) пользователя и фильма  


Оценить RMSE на тестовой выборке

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

from tqdm import tqdm_notebook

import matplotlib.pyplot as plt

from tqdm import tqdm_notebook

from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.neighbors import NearestNeighbors

%matplotlib inline
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestClassifier

from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor

In [2]:
##links = pd.read_csv('links.csv')
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
tags = pd.read_csv('tags.csv')

In [3]:
def change_string(s):
    return ' '.join(s.lower().replace(' ', '').replace('-', '').split('|'))

In [4]:
# Нормализуем жанры
movies['genres_formatted'] = movies.genres.apply(change_string)


In [5]:
movies=movies.drop(['genres'], axis=1)

In [6]:
# слепим теги каждого юзера,  если они относятся к одному фильму
tags['tag_new'] = tags[['userId','movieId','tag']].groupby(['userId','movieId'])['tag'].transform(lambda x: ' '.join(x))
tags=tags[['userId','movieId','tag_new']].drop_duplicates()

### Причешем наши теги

In [7]:
import pymorphy2
from nltk.tokenize import word_tokenize
import nltk
#nltk.download('punkt')
from tqdm import tqdm

In [8]:
class MorphProvider:
    def __init__(self):
        self.cache = {}
        self.morph = pymorphy2.MorphAnalyzer()
    
    def __call__(self, w):
        w = w.lower()
        cached = self.cache.get(w)
        if cached:
            return cached
        try:
            morphed = self.morph.parse(w)[0].normal_form
            self.cache[w] = morphed
            return morphed
        except:
            return None
    
    def morph_string(self, s):
        words = word_tokenize(s)
        return " ".join([self.__call__(w) for w in words])

In [9]:
morph = MorphProvider()

In [10]:
tags["tag_normalized"] = [morph.morph_string(t) for t in tqdm(tags.tag_new)]

100%|██████████| 1775/1775 [00:00<00:00, 8090.59it/s]


In [11]:
tags=tags.drop(['tag_new'], axis=1)

In [12]:
tags.head()

,userId,movieId,tag_normalized
0,2,60756,funny highly quotable will ferrell
3,2,89774,boxing story mma tom hardy
6,2,106782,drugs leonardo dicaprio martin scorsese
9,7,48516,way too long
10,18,431,al pacino gangster mafia


## Сделаем какой-то бейслайн простой:

### Посчитаем разные метрики

In [13]:
from scipy import stats

In [14]:
raitis_aggr= ratings.groupby(['movieId'])[['rating']].count()
raitis_aggr.columns=['count_']
raitis_aggr['mean_']= ratings.groupby(['movieId'])[['rating']].mean()
raitis_aggr['median_']= ratings.groupby(['movieId'])[['rating']].median()
raitis_aggr['var_']= ratings.groupby(['movieId'])[['rating']].var()
raitis_aggr['mode']=ratings.groupby(['movieId']).rating.agg(lambda x: stats.mode(x)[0])

In [15]:
raitis_aggr.shape

(9724, 5)

In [16]:
raitis_aggr.head(4)

,count_,mean_,median_,var_,mode
movieId,,,,,
1,215,3.920930,4.0,0.696990,4.0
2,110,3.431818,3.5,0.777419,4.0
3,52,3.259615,3.0,1.112651,3.0
4,7,2.357143,3.0,0.726190,3.0


In [17]:
ratings[['userId','movieId','rating']].shape

(100836, 3)

In [18]:
x = pd.merge(ratings[['userId','movieId','rating']], raitis_aggr, on='movieId', how='left')

In [19]:
y=x.rating


In [20]:
x=x.drop(['rating'], axis=1)
x=x.drop(['userId'], axis=1)
x=x.drop(['movieId'], axis=1)

In [21]:
x.head()

,count_,mean_,median_,var_,mode
0,215,3.920930,4.0,0.696990,4.0
1,52,3.259615,3.0,1.112651,3.0
2,102,3.946078,4.0,0.667856,4.0
3,203,3.975369,4.0,0.850875,4.0
4,204,4.237745,4.5,0.641475,5.0


In [22]:

x.isnull().any()

count_     False
mean_      False
median_    False
var_        True
mode       False
dtype: bool

In [23]:
#заменим  пропуски в дисперсии на ноль
x.var_  = x.var_.fillna(0)


In [24]:
x_means=x.copy()

### Обучим на этих признаках какой-нибдуь регрессор

In [25]:

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3,random_state=42)

In [26]:
model = RandomForestRegressor()
model.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [27]:


predictions = model.predict(X_test)

print('root_mean_squared_error = ', np.sqrt(mean_squared_error(y_test, predictions)))

root_mean_squared_error =  0.9339128649326034


In [28]:
print(model.feature_importances_)

[0.03433479 0.87907807 0.01203886 0.06046801 0.01408027]


## Попробуем на жанрах обучиться

In [29]:
x = ratings[['userId','movieId','rating']]

In [30]:
movies.head(2)

,movieId,title,genres_formatted
0,1,Toy Story (1995),adventure animation children comedy fantasy
1,2,Jumanji (1995),adventure children fantasy


In [31]:
x = pd.merge(x, movies[['movieId','genres_formatted']], on='movieId', how='left')

In [32]:
x.shape

(100836, 4)

In [33]:
x.head()

,userId,movieId,rating,genres_formatted
0,1,1,4.0,adventure animation children comedy fantasy
1,1,3,4.0,comedy romance
2,1,6,4.0,action crime thriller
3,1,47,5.0,mystery thriller
4,1,50,5.0,crime mystery thriller


In [34]:

x.isnull().any()

userId              False
movieId             False
rating              False
genres_formatted    False
dtype: bool

In [35]:
x=x.drop(['rating'], axis=1)
x=x.drop(['userId'], axis=1)
x=x.drop(['movieId'], axis=1)

In [36]:
cv_genres_ = CountVectorizer()
tf_genres = TfidfTransformer()

In [37]:
cv_genres = cv_genres_.fit_transform(x.genres_formatted)

In [38]:
cv_genres_.get_feature_names()

['action',
 'adventure',
 'animation',
 'children',
 'comedy',
 'crime',
 'documentary',
 'drama',
 'fantasy',
 'filmnoir',
 'horror',
 'imax',
 'musical',
 'mystery',
 'nogenreslisted',
 'romance',
 'scifi',
 'thriller',
 'war',
 'western']

In [39]:
len(cv_genres_.get_feature_names())

20

In [40]:
tfidf_genres= tf_genres.fit_transform(cv_genres)

In [41]:
tfidf_genres

<100836x20 sparse matrix of type '<class 'numpy.float64'>'
	with 274480 stored elements in Compressed Sparse Row format>

In [42]:

X_train, X_test, y_train, y_test = train_test_split(tfidf_genres, y, test_size=0.3,random_state=42)

In [43]:
model = RandomForestRegressor()
model.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [44]:


predictions = model.predict(X_test)

print('root_mean_squared_error = ', np.sqrt(mean_squared_error(y_test, predictions)))

root_mean_squared_error =  1.0061130684106336


In [45]:
print(model.feature_importances_)

[0.09020504 0.0544587  0.0239709  0.08022338 0.12844375 0.08256166
 0.00483693 0.17550205 0.04358117 0.00624612 0.05935347 0.01374025
 0.01232324 0.03332338 0.00056589 0.03447486 0.04928056 0.05383639
 0.03917867 0.01389359]


### Попробуем, нектоорые настройки CountVectorizer

In [46]:
cv_genres_mod_ = CountVectorizer(max_features=10,ngram_range=(1,3))
tf_genres_mod = TfidfTransformer()

In [47]:
cv_genres_mod = cv_genres_mod_.fit_transform(x.genres_formatted)

In [48]:
cv_genres_mod_.get_feature_names()

['action',
 'action adventure',
 'adventure',
 'comedy',
 'crime',
 'drama',
 'fantasy',
 'romance',
 'scifi',
 'thriller']

In [49]:
tfidf_genres_mod= tf_genres_mod.fit_transform(cv_genres)

In [50]:

X_train, X_test, y_train, y_test = train_test_split(tfidf_genres_mod, y, test_size=0.3,random_state=42)

In [51]:
model = RandomForestRegressor()
model.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [52]:


predictions = model.predict(X_test)

print('root_mean_squared_error = ', np.sqrt(mean_squared_error(y_test, predictions)))

root_mean_squared_error =  1.0058448380960276


#### не наблюдаем никакого улучшения, похоже

In [53]:
print(model.feature_importances_)

[0.09750511 0.05255969 0.02690478 0.07653008 0.12693148 0.07946679
 0.00471346 0.17754274 0.03968749 0.00727563 0.05842643 0.01479296
 0.01336197 0.03818986 0.00106172 0.03676173 0.04120655 0.05298555
 0.03920051 0.01489548]


## Попробуем на тегах обучиться

In [54]:
tags.head()

,userId,movieId,tag_normalized
0,2,60756,funny highly quotable will ferrell
3,2,89774,boxing story mma tom hardy
6,2,106782,drugs leonardo dicaprio martin scorsese
9,7,48516,way too long
10,18,431,al pacino gangster mafia


In [55]:
x.head()

,genres_formatted
0,adventure animation children comedy fantasy
1,comedy romance
2,action crime thriller
3,mystery thriller
4,crime mystery thriller


In [56]:
x = ratings[['userId','movieId','rating']]

In [57]:
tags.head()

,userId,movieId,tag_normalized
0,2,60756,funny highly quotable will ferrell
3,2,89774,boxing story mma tom hardy
6,2,106782,drugs leonardo dicaprio martin scorsese
9,7,48516,way too long
10,18,431,al pacino gangster mafia


In [58]:
tags.shape

(1775, 3)

In [59]:
x = pd.merge(x, tags[['movieId','userId','tag_normalized']], on=['userId','movieId'], how='left')

In [60]:
x.shape

(100836, 4)

In [61]:

x.isnull().any()

userId            False
movieId           False
rating            False
tag_normalized     True
dtype: bool

In [62]:
x.tag_normalized = x.tag_normalized.fillna('')

In [63]:
x=x.drop(['rating'], axis=1)
x=x.drop(['userId'], axis=1)
x=x.drop(['movieId'], axis=1)

In [64]:
x.head()

,tag_normalized
0,
1,
2,
3,
4,


In [65]:
x.shape

(100836, 1)

In [66]:
cv_tags = CountVectorizer()
tf_tags = TfidfTransformer()

In [67]:
cv_tags = cv_tags.fit_transform(x.tag_normalized)

In [68]:
tfidf_tags= tf_tags.fit_transform(cv_genres)

In [69]:
tfidf_tags

<100836x20 sparse matrix of type '<class 'numpy.float64'>'
	with 274480 stored elements in Compressed Sparse Row format>

In [70]:

X_train, X_test, y_train, y_test = train_test_split(tfidf_tags, y, test_size=0.3,random_state=42)

In [71]:
model = RandomForestRegressor()
model.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [72]:


predictions = model.predict(X_test)

print('root_mean_squared_error = ', np.sqrt(mean_squared_error(y_test, predictions)))

root_mean_squared_error =  1.0058237731646158


In [73]:
print(model.feature_importances_)

[0.1048162  0.04968774 0.02482919 0.08162478 0.12580801 0.08127558
 0.00489484 0.17385679 0.0438361  0.00798671 0.05951987 0.01343584
 0.01127452 0.04243824 0.00039009 0.03455844 0.0392434  0.0470803
 0.03915518 0.01428817]


## Гм, а если на тех и на других призанках?

In [74]:
X_train, X_test, y_train, y_test = train_test_split(np.hstack((tfidf_tags.toarray(),tfidf_genres.toarray())), y, test_size=0.3,random_state=42)

In [75]:
model = RandomForestRegressor()
model.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [76]:


predictions = model.predict(X_test)

print('root_mean_squared_error = ', np.sqrt(mean_squared_error(y_test, predictions)))

root_mean_squared_error =  1.0059847864493903


In [77]:
print(model.feature_importances_)

[5.71091410e-02 2.25656384e-02 1.31496488e-02 4.24993103e-02
 4.62716310e-02 3.63683497e-02 3.10297436e-03 5.40866245e-02
 2.96012228e-02 3.10145832e-03 3.04949870e-02 7.05513944e-03
 5.44219950e-03 2.03386958e-02 5.77792058e-04 1.90844981e-02
 2.35986709e-02 2.80429652e-02 1.50992514e-02 7.39671195e-03
 4.71323314e-02 2.88404859e-02 1.23882884e-02 3.38694633e-02
 6.94963998e-02 4.34503788e-02 3.86544735e-03 1.17018145e-01
 1.56820386e-02 3.62719719e-03 3.27005272e-02 5.54785758e-03
 7.75125303e-03 1.49229987e-02 4.25890150e-08 1.77546681e-02
 2.24766267e-02 2.54069046e-02 2.58575701e-02 7.22446482e-03]


In [78]:
x_means.values.shape

(100836, 5)

In [79]:
y.values

array([4., 4., 4., ..., 5., 5., 3.])

### Добавим еще наши средние

In [80]:
X_train, X_test, y_train, y_test = train_test_split(np.hstack((tfidf_tags.toarray(),tfidf_genres.toarray(),x_means.values)), y.values, test_size=0.3,random_state=42)

In [81]:
model = RandomForestRegressor()
model.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [82]:


predictions = model.predict(X_test)

print('root_mean_squared_error = ', np.sqrt(mean_squared_error(y_test, predictions)))

root_mean_squared_error =  0.9473873775241615


In [83]:
print(model.feature_importances_)

[3.53997218e-03 2.76691659e-03 1.29371020e-03 1.63625179e-03
 5.08317120e-03 2.31749797e-03 6.77740559e-04 5.04980897e-03
 1.86579155e-03 2.13106340e-04 2.17883184e-03 7.72352988e-04
 1.08553188e-03 1.41187652e-03 1.67777842e-05 3.11017331e-03
 2.49709997e-03 3.04442387e-03 1.22268138e-03 5.09376438e-04
 3.54558253e-03 2.90406968e-03 1.14739076e-03 1.61034771e-03
 4.98439889e-03 2.39154298e-03 4.67850102e-04 5.13973948e-03
 1.84760233e-03 2.83533946e-04 2.11199780e-03 6.05660264e-04
 9.37412189e-04 1.40406083e-03 4.47550547e-05 2.92437894e-03
 2.23600129e-03 2.96484583e-03 9.65691162e-04 5.39372314e-04
 1.90758428e-02 8.44284968e-01 7.92019674e-03 3.92491683e-02
 1.01204968e-02]


###  И так мы тоже не превзоошли бейс-лайн

In [84]:
%%time
lr_params = {
    'fit_intercept':[False, True]
}

#fit_intercepts = [False, True]
#param_regr = dict(fit_intercept=fit_intercepts)
lr = LinearRegression()
grid_lr = GridSearchCV(lr, lr_params,
                       scoring='neg_mean_squared_error',
                       cv=3,n_jobs=-1)
grid_lr.fit(X_train, y_train)

print(grid_lr.best_params_)
print(grid_lr.best_score_)
print(grid_lr.best_estimator_)

{'fit_intercept': False}
-0.767318450578771
LinearRegression(copy_X=True, fit_intercept=False, n_jobs=None,
         normalize=False)
CPU times: user 285 ms, sys: 313 ms, total: 598 ms
Wall time: 1.5 s


In [85]:
#Оценим тчоностьна тесте
grid_lr.score(X_test,y_test)

-0.770282928393519

In [86]:

print('root_mean_squared_error = ', np.sqrt(mean_squared_error(y_test, grid_lr.best_estimator_.predict(X_test))))

root_mean_squared_error =  0.8776576373470004


In [87]:
np.sqrt(-grid_lr.score(X_test,y_test))

0.8776576373470004

In [88]:
RandomizedSearchCV

sklearn.model_selection._search.RandomizedSearchCV

In [89]:
%%time

knn_params = {
    'n_neighbors':list(range(1, 30))
   ,'weights': ['uniform', 'distance']
   ,'algorithm' : ['auto', 'ball_tree', 'kd_tree', 'brute']
   ,'leaf_size':list(range(5, 30))
}

grid_knn = RandomizedSearchCV(KNeighborsRegressor(), knn_params,  scoring='neg_mean_squared_error',
                       cv=3,n_jobs=-1)
grid_knn.fit(X_train, y_train)
print(grid_knn.best_params_)
print(grid_knn.best_score_)
print(grid_knn.best_estimator_)

{'weights': 'uniform', 'n_neighbors': 28, 'leaf_size': 18, 'algorithm': 'auto'}
-0.8206943963566867
KNeighborsRegressor(algorithm='auto', leaf_size=18, metric='minkowski',
          metric_params=None, n_jobs=None, n_neighbors=28, p=2,
          weights='uniform')
CPU times: user 418 ms, sys: 401 ms, total: 819 ms
Wall time: 2min 24s


In [90]:
grid_knn.score(X_test,y_test)

-0.8226370721257992

In [91]:

print('root_mean_squared_error = ', np.sqrt(mean_squared_error(y_test, grid_knn.best_estimator_.predict(X_test))))

root_mean_squared_error =  0.9069934245218094


In [92]:
%%time
dt_params = {
    'max_depth':[None,1,2,5,10,25,50],
     'min_samples_split':[2,5,8,10,25,50],
    
    'min_weight_fraction_leaf': [0, 0.01, 0.1, 0.15, 0.25, 0.5] ,
    'min_samples_leaf':list(range(1, 10)),
     'criterion':  ['mse', 'friedman_mse', 'mae'],
    'max_features':list(range(1, 13)) 
}
#model_forest = RandomForestRegressor(
    

grid_dt =  RandomizedSearchCV(DecisionTreeRegressor(),dt_params,  scoring='neg_mean_squared_error',
                       cv=3,n_jobs=-1)
grid_dt.fit(X_train, y_train)
print(grid_dt.best_params_)
print(grid_dt.best_score_)
print(grid_dt.best_estimator_)

{'min_weight_fraction_leaf': 0.01, 'min_samples_split': 50, 'min_samples_leaf': 9, 'max_features': 6, 'max_depth': None, 'criterion': 'mse'}
-0.8249299984723701
DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=6,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=9,
           min_samples_split=50, min_weight_fraction_leaf=0.01,
           presort=False, random_state=None, splitter='best')
CPU times: user 134 ms, sys: 27 ms, total: 162 ms
Wall time: 1min 7s


In [93]:

grid_dt.score(X_test,y_test)

-0.8528312558569267

In [94]:

print('root_mean_squared_error = ', np.sqrt(mean_squared_error(y_test, grid_dt.best_estimator_.predict(X_test))))

root_mean_squared_error =  0.9234886333122496


In [95]:
%%time
rf_params = {
    'n_estimators': [1,10,20,30,40,50,60,80,90],
    
    'max_depth':[None,1,2,5,10,25,50],
    
    'min_samples_leaf':list(range(1, 10)),
   
    'max_features':list(range(1, 13)) ,
     'criterion':  ['mse', 'friedman_mse', 'mae']
}

    

grid_rf =  RandomizedSearchCV(RandomForestRegressor(),rf_params,scoring='neg_mean_squared_error',
                       cv=3,n_jobs=-1)
grid_rf.fit(X_train, y_train)
print(grid_rf.best_params_)
print(grid_rf.best_score_)
print(grid_rf.best_estimator_)

{'n_estimators': 60, 'min_samples_leaf': 4, 'max_features': 6, 'max_depth': 10, 'criterion': 'mae'}
-0.8070001628257499
RandomForestRegressor(bootstrap=True, criterion='mae', max_depth=10,
           max_features=6, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=4,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=60, n_jobs=None, oob_score=False,
           random_state=None, verbose=0, warm_start=False)
CPU times: user 29min 55s, sys: 295 ms, total: 29min 55s
Wall time: 56min 17s


In [96]:
grid_rf.score(X_test,y_test)

-0.8110447307297463

In [97]:

print('root_mean_squared_error = ', np.sqrt(mean_squared_error(y_test, grid_rf.best_estimator_.predict(X_test))))

root_mean_squared_error =  0.9005802189309657


In [99]:
%%time
from sklearn.ensemble.gradient_boosting import GradientBoostingRegressor
gb_params = {
          'n_estimators': [1,10,20,30,40,50,60,80,90],
          'max_features': list(range(1, 13)),
             'max_depth':[None,1,2,5,10,25,50],
            'learning_rate': [0.1,0.3,0.5,0.7],
               #'min_samples_split':[2,5,8,10,25,50],
            'min_samples_leaf':list(range(1, 10)),
}

    

grid_gb =  RandomizedSearchCV(GradientBoostingRegressor(),gb_params,scoring='neg_mean_squared_error',
                       cv=3,n_jobs=-1)
grid_gb.fit(X_train, y_train)
print(grid_gb.best_params_)
print(grid_gb.best_score_)
print(grid_gb.best_estimator_)

{'n_estimators': 20, 'min_samples_leaf': 8, 'max_features': 10, 'max_depth': 5, 'learning_rate': 0.1}
-0.785112992231112
GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=5, max_features=10,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=8,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=20, n_iter_no_change=None, presort='auto',
             random_state=None, subsample=1.0, tol=0.0001,
             validation_fraction=0.1, verbose=0, warm_start=False)
CPU times: user 1.57 s, sys: 96 ms, total: 1.67 s
Wall time: 2min 57s


In [100]:
grid_gb.score(X_test,y_test)

-0.790124750250296

In [101]:

print('root_mean_squared_error = ', np.sqrt(mean_squared_error(y_test, grid_gb.best_estimator_.predict(X_test))))

root_mean_squared_error =  0.8888896164599382


In [102]:
%%time
SVR_params = {
          #'kernel':['linear', 'poly','rbf', 'sigmoid', 'precomputed'],
             'C' :[0.001, 0.01, 0.1, 1, 10],
    'gamma': [0.001, 0.01, 0.1, 1]
}


grid_SVR =   GridSearchCV(SVR(),SVR_params,scoring='neg_mean_squared_error',
                       cv=3,n_jobs=-1)
grid_SVR.fit(X_train, y_train)
print(grid_SVR.best_params_)
print(grid_SVR.best_score_)
print(grid_SVR.best_estimator_)

{'C': 1, 'gamma': 0.001}
-0.791743290862153
SVR(C=1, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.001,
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)
CPU times: user 5min 22s, sys: 128 ms, total: 5min 22s
Wall time: 1h 28min 33s


In [103]:

grid_SVR.score(X_test,y_test)

-0.7950338017723689

In [104]:

print('root_mean_squared_error = ', np.sqrt(mean_squared_error(y_test, grid_SVR.best_estimator_.predict(X_test))))

root_mean_squared_error =  0.8916466798975752


In [108]:
model=['SVR','gb','dt','rf','knn','lr']
#model=['lr']

In [115]:
for model_ in models:
    print(model_,'---','root_mean_squared_error = ', np.sqrt(mean_squared_error(y_test, eval('grid_'+model_+'.best_estimator_').predict(X_test))))

SVR --- root_mean_squared_error =  0.8916466798975752
gb --- root_mean_squared_error =  0.8888896164599382
dt --- root_mean_squared_error =  0.9234886333122496
knn --- root_mean_squared_error =  0.9069934245218094
lr --- root_mean_squared_error =  0.8776576373470004


### Ну вот даже улучшили немого результат